In [7]:
import pandas as pd
import numpy as np
import warnings
import re

warnings.filterwarnings('ignore')

# --- 1. KONFIGURASI FILE & MAPPING ---
FILE_SUMMARY = "Summary_Cabang_AB_2025 (Split).xlsx"
FILE_MASTER = "cost & bbm 2022 sd 2025 HP & Type.xlsx"
FILE_BBM = "BBM AAB.xlsx"

MAP_CABANG = {
    'TANGKIANG': 'CABANG LUWUK', 'AMBON': 'CABANG AMBON', 'BAU BAU': 'BAU-BAU',
    'MANOKWARI': 'DEPO MANOKWARI', 'MERAK': 'DEPO MERAK', 'MARUNI': 'DEPO MARUNI'
}

STATUS_WEIGHT_TYPE = {
    'MTA': 'empty', 'STR': 'full', 'MTB': 'full', 'STF': 'full', 'FAC': 'full', 'MAS': 'empty', 
    'FOB': 'full', 'MOB': 'empty', 'FXD': 'full', 'MXD': 'empty', 'FTL': 'full', 'MTL': 'empty', 'FIT': 'full', 'MIT': 'empty'
}

WEIGHT_DICT = {
    ('20 Feet', 'empty'): 2300, ('20 Feet', 'full'): 27000,
    ('40 Feet', 'empty'): 3800, ('40 Feet', 'full'): 32000
}

EXCLUDED_BRANCHES = ['JAKARTA', 'SURABAYA']

def clean_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    name = name.replace("FORKLIFT", "FORKLIF")
    return re.sub(r'[^A-Z0-9]', '', name)

# --- 2. BACA MASTER DATA DARI COST & BBM (SATU-SATUNYA SUMBER KEBENARAN) ---
print("Loading Master Data...")
master_data_map = {} 
master_keys_set = set()

df_map = pd.read_excel(FILE_MASTER, sheet_name='Sheet2', header=1)
col_name = next((c for c in df_map.columns if 'NAMA' in str(c).upper()), None)
col_jenis = next((c for c in df_map.columns if 'ALAT' in str(c).upper() and 'BERAT' in str(c).upper() and c != col_name), None)
col_type = next((c for c in df_map.columns if 'TYPE' in str(c).upper() or 'MERK' in str(c).upper()), None)
col_hp = next((c for c in df_map.columns if any(k == str(c).upper() for k in ['HP', 'HORSE POWER'])), None)
col_cap = next((c for c in df_map.columns if any(k in str(c).upper() for k in ['CAP', 'KAPASITAS'])), None)
col_loc = next((c for c in df_map.columns if 'LOKASI' in str(c).upper() or 'DES 2025' in str(c).upper()), df_map.columns[2])

df_map.dropna(subset=[col_name], inplace=True)
df_map['Unit_Original'] = df_map[col_name].astype(str).str.strip().str.upper()
df_map = df_map[~df_map['Unit_Original'].str.contains('DUMMY', na=False)]
df_map = df_map[~df_map['Unit_Original'].str.contains('FALCON', na=False)]

for _, row in df_map.iterrows():
    u_name = str(row['Unit_Original'])
    c_id = clean_unit_name(u_name)
    if c_id:
        cap_val = 0
        try:
            match = re.search(r"(\d+(\.\d+)?)", str(row[col_cap]))
            if match: cap_val = int(float(match.group(1)) + 0.5)
        except: pass

        t_merk = str(row[col_type]).strip().upper() if pd.notna(row[col_type]) else "-"
        jenis = str(row[col_jenis]).strip().upper() if pd.notna(row[col_jenis]) else "-"
        is_trucking = True if any(x in jenis for x in ['TRAILER', 'TRONTON', 'TRUCK']) else False

        master_data_map[c_id] = {
            'Unit_Name': u_name, 'Jenis_Alat': jenis, 'Type_Merk': t_merk, 
            'Horse_Power': row[col_hp] if pd.notna(row[col_hp]) else "0", 
            'Capacity': cap_val, 'Lokasi': str(row[col_loc]).strip().upper(),
            'Is_Trucking': is_trucking
        }
        master_keys_set.add(c_id)

# --- 3. PEMBACAAN DATA BBM (CERDAS MENDETEKSI PERUBAHAN NAMA & EX) ---
print("Loading BBM AAB & Mapping Names...")
sheets_bbm = {'JAN': 'Januari', 'FEB': 'Februari', 'MAR': 'Maret', 'APR': 'April', 'MEI': 'Mei', 'JUN': 'Juni', 'JUL': 'Juli', 'AGT': 'Agustus', 'SEP': 'September', 'OKT': 'Oktober', 'NOV': 'November'}

list_df_bbm = []
xls = pd.ExcelFile(FILE_BBM)

def get_master_match(raw_name):
    raw_name = str(raw_name).strip().upper()
    
    # PENAMBAHAN MANUAL MAPPING SESUAI PERMINTAAN
    if "WIND RIVER" in raw_name:
        return clean_unit_name("TOP LOADER BOSS") if clean_unit_name("TOP LOADER BOSS") in master_data_map else None
    
    if "FL RENTAL 01" in raw_name and "TIMIKA" not in raw_name:
        return clean_unit_name("FL RENTAL 01 TIMIKA") if clean_unit_name("FL RENTAL 01 TIMIKA") in master_data_map else None
    if "TOBATI" in raw_name:
        return clean_unit_name("TOP LOADER KALMAR 35 T/TOBATI") if clean_unit_name("TOP LOADER KALMAR 35 T/TOBATI") in master_data_map else None
    if "L 8477 UUC" in raw_name:
        return clean_unit_name("L 9902 UR / S75") if clean_unit_name("L 9902 UR / S75") in master_data_map else None
        
    c_raw = clean_unit_name(raw_name)
    if c_raw in master_data_map: return c_raw
        
    if " (" in raw_name:
        b_paren = clean_unit_name(raw_name.split(" (")[0])
        if b_paren in master_data_map: return b_paren
            
    if "EX." in raw_name or "EX " in raw_name:
        after_ex = raw_name.split("EX.")[-1] if "EX." in raw_name else raw_name.split("EX ")[-1]
        c_after = clean_unit_name(after_ex.replace(")", ""))
        if c_after in master_data_map: return c_after
        for m_key in master_keys_set:
            if c_after != "" and c_after in m_key: return m_key
                
    for p in raw_name.replace("(", "/").replace(")", "/").split("/"):
        cl = clean_unit_name(p)
        if cl in master_data_map: return cl
    return None

for sht, bln in sheets_bbm.items():
    if sht in xls.sheet_names:
        df_temp = pd.read_excel(xls, sheet_name=sht, header=None)
        unit_names_row = df_temp.iloc[0].ffill()
        headers = df_temp.iloc[2]
        data = df_temp.iloc[3:]
        
        for col in range(1, df_temp.shape[1]):
            header_str = str(headers[col]).strip().upper()
            if header_str in ['LITER', 'QTY']:
                raw_unit_name = str(unit_names_row[col]).strip().upper()
                if raw_unit_name == "" or "UNNAMED" in raw_unit_name or "TOTAL" in raw_unit_name: continue
                if raw_unit_name.startswith(('GENSET', 'KOMPRESSOR', 'MESIN', 'TANGKI', 'SPBU', 'MOBIL', 'GROUP')): continue
                
                matched_id = get_master_match(raw_unit_name)
                if matched_id:
                    vals = pd.to_numeric(data[col], errors='coerce').sum()
                    if vals > 0:
                        list_df_bbm.append({'Unit_Name': master_data_map[matched_id]['Unit_Name'], 'Bulan': bln, 'LITER': vals})

df_bbm_all = pd.DataFrame(list_df_bbm).groupby(['Unit_Name', 'Bulan'])['LITER'].sum().reset_index() if list_df_bbm else pd.DataFrame()
if not df_bbm_all.empty:
    df_bbm_all.to_excel("Rekap_BBM_All.xlsx", index=False)

# --- 4. PERHITUNGAN TONASE ALAT BERAT ---
def get_allowed_tasks(jenis, cap):
    jenis = str(jenis).upper()
    if 'FORKLIFT' in jenis:
        if cap == 0 or (3 <= cap <= 8): return ['STR', 'STF', 'MTB']
        elif cap >= 10: return ['STR', 'STF', 'MTB', 'FAC', 'MAS'] 
    elif 'REACH STACKER' in jenis: return ['FOB', 'MOB', 'FXD', 'MXD', 'FAC', 'MAS']
    elif 'TOP LOADER' in jenis or 'SIDE LOADER' in jenis: return ['FOB', 'MOB', 'FXD', 'MXD', 'FAC', 'MAS']
    elif 'CRANE' in jenis:
        if cap >= 70: return ['FOB', 'MOB', 'FXD', 'MXD']
        elif cap >= 40: return ['FOB', 'MOB', 'FXD', 'MXD', 'FAC', 'MAS']
    return []

print("Loading Summary Cabang...")
xls_summary = pd.ExcelFile(FILE_SUMMARY)
all_cabang_data = []

for sheet in xls_summary.sheet_names:
    cabang_summary = str(sheet).upper().strip()
    if cabang_summary in EXCLUDED_BRANCHES: continue 
        
    lokasi_bbm = MAP_CABANG.get(cabang_summary, cabang_summary)
    unit_cabang_dict = {k:v for k,v in master_data_map.items() if not v['Is_Trucking'] and clean_unit_name(lokasi_bbm) in clean_unit_name(str(v['Lokasi']))}
    if not unit_cabang_dict: continue 
        
    try:
        df_sheet = pd.read_excel(FILE_SUMMARY, sheet_name=sheet, header=2).dropna(subset=['Bulan'])
        df_sheet = df_sheet[df_sheet['Bulan'] != 'Grand Total']
        raw_df = pd.read_excel(FILE_SUMMARY, sheet_name=sheet, header=None)
        clean_status = [str(x).split(' ')[0].upper() for x in raw_df.iloc[2].fillna(method='ffill').values]
        header_size = raw_df.iloc[3].values
        
        for idx, row in df_sheet.iterrows():
            bulan = str(row['Bulan']).strip().capitalize()
            task_counts = {}
            for col_idx in range(1, len(row)):
                if col_idx >= len(clean_status): break
                stat, size, val = clean_status[col_idx], str(header_size[col_idx]).strip(), row.iloc[col_idx]
                if pd.notna(val) and val != '' and stat != 'TOTAL':
                    task_counts[(stat, size)] = task_counts.get((stat, size), 0) + float(val)
                    
            for u_id, unit in unit_cabang_dict.items():
                allowed = get_allowed_tasks(unit['Jenis_Alat'], unit['Capacity'])
                total_berat = 0
                for (stat, size), count in task_counts.items():
                    if stat in allowed:
                        rekan = sum(1 for k, u_lain in unit_cabang_dict.items() if stat in get_allowed_tasks(u_lain['Jenis_Alat'], u_lain['Capacity']))
                        if rekan > 0: total_berat += (count / rekan) * WEIGHT_DICT.get((size, STATUS_WEIGHT_TYPE.get(stat, 'empty')), 0)
                
                if total_berat > 0:
                    all_cabang_data.append({'Unit_Name': unit['Unit_Name'], 'Bulan': bulan, 'Total_Ton': total_berat / 1000})
    except: pass

# --- 5. MERGE DATA FINAL (NON-TRUCKING SAJA) ---
df_tonase = pd.DataFrame(all_cabang_data).groupby(['Unit_Name', 'Bulan'])['Total_Ton'].sum().reset_index() if all_cabang_data else pd.DataFrame(columns=['Unit_Name', 'Bulan', 'Total_Ton'])
df_bbm_ab = df_bbm_all[df_bbm_all['Unit_Name'].isin([v['Unit_Name'] for v in master_data_map.values() if not v['Is_Trucking']])] if not df_bbm_all.empty else pd.DataFrame(columns=['Unit_Name', 'Bulan', 'LITER'])

df_monthly = pd.merge(df_tonase, df_bbm_ab, on=['Unit_Name', 'Bulan'], how='outer').fillna(0)
for k in ['Jenis_Alat', 'Type_Merk', 'Horse_Power', 'Capacity', 'Lokasi']:
    df_monthly[k] = df_monthly['Unit_Name'].apply(lambda n: master_data_map[clean_unit_name(n)][k])

df_monthly['Fuel_Ratio'] = np.where(df_monthly['Total_Ton'] > 0, df_monthly['LITER'] / df_monthly['Total_Ton'], 0)
df_total = df_monthly.groupby(['Unit_Name', 'Lokasi', 'Jenis_Alat', 'Type_Merk', 'Horse_Power', 'Capacity']).agg({'Total_Ton': 'sum', 'LITER': 'sum'}).reset_index()

# Ekstraksi Unit Inaktif (Alat Berat di Master yang tidak ada aktivitas/BBM)
active_names = df_total['Unit_Name'].unique()
inaktif_list = []
for v in master_data_map.values():
    if not v['Is_Trucking'] and v['Unit_Name'] not in active_names:
        inaktif_list.append({'Nama Unit': v['Unit_Name'], 'Jenis': v['Jenis_Alat'], 'Type/Merk': v['Type_Merk'], 'Lokasi': v['Lokasi'], 'Total Pengisian BBM': 0, 'Total Berat Angkutan (Ton)': 0, 'Keterangan': 'Tidak ada aktivitas (BBM 0 & Tonase 0)'})

# --- 6. EXPORT FILE ---
with pd.ExcelWriter("HasilNonTrucking.xlsx") as writer:
    df_total.to_excel(writer, sheet_name='Total_Agregat', index=False)
    df_monthly.to_excel(writer, sheet_name='Data_Bulanan', index=False)
    if inaktif_list: pd.DataFrame(inaktif_list).to_excel(writer, sheet_name='Unit_Inaktif', index=False)

df_trend = df_monthly.copy()
map_bln = {'Januari': '2025-01', 'Februari': '2025-02', 'Maret': '2025-03', 'April': '2025-04', 'Mei': '2025-05', 'Juni': '2025-06', 'Juli': '2025-07', 'Agustus': '2025-08', 'September': '2025-09', 'Oktober': '2025-10', 'November': '2025-11'}
df_trend['Bulan_Num'] = df_trend['Bulan'].map(map_bln)
df_pivot = df_trend.pivot_table(index=['Jenis_Alat', 'Capacity', 'Unit_Name'], columns='Bulan_Num', values='Fuel_Ratio', aggfunc='sum').reset_index().fillna(0)
df_pivot['Status_Tren'] = df_pivot.apply(lambda r: "MEMBAIK (Makin Irit)" if r.get('2025-11',0) < r.get('2025-01',0)*0.95 else ("MEMBURUK (Makin Boros)" if r.get('2025-11',0) > r.get('2025-01',0)*1.05 else "STABIL"), axis=1)
df_pivot.to_excel("Laporan_Tren_Efisiensi_Bulanan_NonTrucking.xlsx", index=False)

print("Selesai! Seluruh unit sukses dipetakan dan diekstrak.")

Loading Master Data...
Loading BBM AAB & Mapping Names...
Loading Summary Cabang...
Selesai! Seluruh unit sukses dipetakan dan diekstrak.
